In [1]:
import numpy as np
import pandas as pa
import pickle
from datetime import datetime
import librosa.display
from matplotlib import pyplot as plt
from sklearn.metrics import recall_score, precision_score

from src.utils import get_framed_label, train_test_split, from_boolean_array_to_intervals
from src.data import load_annotation
from src.data import load_radar, load_water_distance, load_weight_sensor, load_audio
from src import make_dataset

import warnings
warnings.filterwarnings('ignore')


with open('../models/trained_models/urination-rf-sources-extended-embedding-0202.pkl', 'rb') as f:
    urinate_model = pickle.load(f)

with open('../models/trained_models/defecation-rf-sources-extended-embedding-0202.pkl', 'rb') as f:
    defecate_model = pickle.load(f)

In [2]:
config = {
    'USE_IDS': [],
    'DATAFRAME_PATH': "C:/Users/Jiajun/Desktop/download-project/data/raw/data_frames",
    'ANNOTATION_PATH': "C:/Users/Jiajun/Desktop/download-project/data/processed/Annotation.csv",
    'FEATURE_NAMES': ['Max', 'Min', 'Mean', 'Median', 'LogVariance', 'LinearTrend'],
    'SOURCE_NAMES': ['TotalWeight', 'RadarSum', 'AudioDelay4'],
    'WINDOW_SECONDS': 2,
    'HOP_SECONDS': 1,
    'CATEGORY': '',
}

annotations = load_annotation.get_annotation(config['ANNOTATION_PATH'])

In [3]:
# parameters
# eval_use_ids
# annotations
# urinate, defecate model

In [19]:
urinate_train_config = {
    'USE_IDS': [],
    'DATAFRAME_PATH': "C:/Users/Jiajun/Desktop/download-project/data/raw/data_frames",
    'ANNOTATION_PATH': "C:/Users/Jiajun/Desktop/download-project/data/processed/Annotation.csv",
    'FEATURE_NAMES': ['Max', 'Min', 'Mean', 'Median', 'LogVariance', 'LinearTrend'],
    'SOURCE_NAMES': ['TotalWeight', 'RadarSum', 'AudioDelay4'],
    'WINDOW_SECONDS': 2,
    'HOP_SECONDS': 1,
    'CATEGORY': 'Urination'
}

defecate_train_config = {
    'USE_IDS': [],
    'DATAFRAME_PATH': "C:/Users/Jiajun/Desktop/download-project/data/raw/data_frames",
    'ANNOTATION_PATH': "C:/Users/Jiajun/Desktop/download-project/data/processed/Annotation.csv",
    'FEATURE_NAMES': ['Max', 'Min', 'Mean', 'Median', 'LogVariance', 'LinearTrend'],
    'SOURCE_NAMES': ['TotalWeight', 'RadarSum', 'AudioDelay4'],
    'WINDOW_SECONDS': 2,
    'HOP_SECONDS': 1,
    'CATEGORY': "Defecation"    
}

eval_config = {
    'USE_IDS': [1802,1806,1808,1818,1826,1828,1829,1830,1831,1832,1833,1834,1835,1836,1839,1841],
    'URINATE_MODEL': urinate_model,
    'DEFECATE_MODEL': defecate_model,
    'URINATE_TRAIN_CONFIG': urinate_train_config,
    'DEFECATE_TRAIN_CONFIG': defecate_train_config,
    'URINATE_THRESHOLD': 0.3,
    'DEFECATE_THRESHOLD': 0.3,
    'ANNOTATION_PATH': 'C:/Users/Jiajun/Desktop/download-project/data/processed/Annotation.csv'
}

In [ ]:
timestamp = datetime.now().strftime("%H-%M-%S")

eval_plot_matrix = EvalPlotMatrix(eval_config)
eval_plot_matrix.plots(f'eval-plot-matrices-{timestamp}.pdf')

In [11]:
class EvalPlotMatrix:
    def __init__(self, eval_config):
        self.eval_use_ids = eval_config['USE_IDS']
        self.urinate_model = eval_config['URINATE_MODEL']
        self.defecate_model = eval_config['DEFECATE_MODEL']
        self.urinate_train_config = eval_config['URINATE_TRAIN_CONFIG']
        self.defecate_train_config = eval_config['DEFECATE_TRAIN_CONFIG']
        self.urinate_threshold = eval_config['URINATE_THRESHOLD']
        self.defecate_threshold = eval_config['DEFECATE_THRESHOLD']
        self.annotations = load_annotation.get_annotation(eval_config['ANNOTATION_PATH'])
        
    def get_x_and_y(self, use_i, category):
        if category == "Urination":
            config = self.urinate_train_config.copy()
        elif category == "Defecation":
            config = self.defecate_train_config.copy()
        config['USE_IDS'] = [use_i]
        dataset_i = make_dataset.RandomForestExtended(config)
        x_i, y_i = dataset_i.get_features_and_labels_from_users()
        return x_i, y_i
    
    def get_predicted_region(self, use_i, category):
        if category == "Urination":
            model = self.urinate_model
            threshold = self.urinate_threshold
        elif category == "Defecation":
            model = self.defecate_model
            threshold = self.defecate_threshold
        self.x_i, self.y_i = self.get_x_and_y(use_i, category)
        self.boolean_array_i = (model.predict_proba(self.x_i)[:, 1] > threshold).astype(int)
        predicted_region = from_boolean_array_to_intervals(self.boolean_array_i)
        return predicted_region
    
    def get_annotated_region(self, use_i, category):
        annotated_region = []
        for region in self.annotations[use_i]:
            if region[-1] == category:
                annotated_region.append(region[:2])
        return annotated_region
    
    def get_eval_statistics(self):
        recall_ = recall_score(y_true=self.y_i, y_pred=self.boolean_array_i)
        precision_ = precision_score(y_true=self.y_i, y_pred=self.boolean_array_i)
        return recall_, precision_
        
    def make_subplot_1(self, ax, use_i, caption):
        total_weight_i = load_weight_sensor.get_total_weight_clean(use_i)
        ax.plot(total_weight_i)
        ax.set_ylim(total_weight_i.median()-1, total_weight_i.median()+1)
        ax.title.set_text(f'annotated: {use_i}; Urine recall:{caption[0]:.2f}, precision:{caption[1]:.2f}')
        for region in self.urinate_annotated_region:
            ax.axvspan(region[0], region[1], color="gold", alpha=0.8)
        for region in self.defecate_annotated_region:
            ax.axvspan(region[0], region[1], color="red", alpha=0.5)
        #ax.figtext(0.5, 0.01,
        #           f'Urine recall:{caption[0]}, precision:{caption[1]}',
        #           wrap=True, horizontalalignment='center', fontsize=12
        #)

    def make_subplot_2(self, ax, use_i, caption):
        total_weight_i = load_weight_sensor.get_total_weight_clean(use_i)
        ax.plot(total_weight_i)
        ax.title.set_text(f'predicted: {use_i}; Stool recall:{caption[0]:.2f}, precision:{caption[1]:.2f}')
        ax.set_ylim(total_weight_i.median()-1, total_weight_i.median()+1)
        for region in self.urinate_predicted_region:
            ax.axvspan(region[0], region[1], color="gold", alpha=0.8)
        for region in self.defecate_predicted_region:
            ax.axvspan(region[0], region[1], color="red", alpha=0.5)
        #ax.figtext(0.5, 0.01,
        #           f'Stool recall:{caption[0]}, precision:{caption[1]}',
        #           wrap=True, horizontalalignment='center', fontsize=12
        #)
            
    def plots(self, filename):
        nrow = len(self.eval_use_ids)
        ncol = 2
        fig, axes = plt.subplots(nrow, ncol, figsize=(10*ncol, 2*nrow))
        for idx, use_i in enumerate(self.eval_use_ids):
            self.urinate_annotated_region = self.get_annotated_region(use_i, "Urination")
            self.urinate_predicted_region = self.get_predicted_region(use_i, "Urination")
            self.urinate_recall, self.urinate_precision = self.get_eval_statistics()

            self.defecate_annotated_region = self.get_annotated_region(use_i, "Defecation")
            self.defecate_predicted_region = self.get_predicted_region(use_i, "Defecation")
            self.defecate_recall, self.defecate_precision = self.get_eval_statistics()

            # axes[idx][0]: total_weight with annotations
            self.make_subplot_1(axes[idx][0], use_i, [self.urinate_recall, self.urinate_precision])            
            # axes[idx][1]: total_weight with prediction
            self.make_subplot_2(axes[idx][1], use_i, [self.defecate_recall, self.defecate_precision])
            
        fig.tight_layout()
        plt.savefig(f'../reports/{filename}')